# Clean dataframe from HATCH Version 1.5
## Country Level Data

First Run: "(1) Clean HATCH Dataset - IIAC And Other Uses" jupyter notebook

#### This notebook changes country names for alignment
- Checks data cleanliness
- Converts country names to ISO3c codes
- annual to cumulative values

In [3]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os.path
from countrycode import countrycode

from matplotlib.pyplot import cm

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


#### Import data 

In [4]:
#Import data from cleaned CSV

path_name = "clean_HATCHv1.5_internal.csv"
all_tech_country = pd.read_csv(path_name, low_memory = False)

# Drop "Unnamed:0" if it exists.
try: 
    all_tech_country = all_tech_country.drop('Unnamed: 0', axis=1)
except:
    print("Columns are consistent.")

all_tech_country

Columns are consistent.


,ID,Spatial Scale,Region,Country Name,Technology Name,Metric,Unit,Data Source,Variable,1700,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Sugar Output_Annual Production_JM,National,JM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sugar Output_Annual Production_BB,National,BB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sugar Output_Annual Production_CU,National,CU,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beer_Annual Production_GB,National,GB,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sugar Output_Annual Production_MU,National,MU,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,Crop Harvester_Total Number_UZ,National,UZ,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6133,Television_Total Number_BA,National,BA,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6134,BCG Vaccine_Share of Population_AT,National,AT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6135,MCV2 Vaccine_Share of Population_DJ,National,DJ,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose,NaN,...,0.81,0.81,0.81,0.81,0.6,0.48,NaN,NaN,NaN,NaN


### Drop Non-Relevant Metrics

Costs, cumulative rated capacity (duplicates from storage technologies), and any typos 

In [5]:
# Drop non-relevant metrics (cost, duplicate measures for storage)
all_tech_country.drop(all_tech_country[all_tech_country['Metric'] == "Cumulative Rated Capacity"].index, inplace = True)

all_tech_country = all_tech_country[~all_tech_country['Metric'].str.contains('Cost')]

all_tech_country

,ID,Spatial Scale,Region,Country Name,Technology Name,Metric,Unit,Data Source,Variable,1700,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Sugar Output_Annual Production_JM,National,JM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sugar Output_Annual Production_BB,National,BB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sugar Output_Annual Production_CU,National,CU,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beer_Annual Production_GB,National,GB,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sugar Output_Annual Production_MU,National,MU,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,Crop Harvester_Total Number_UZ,National,UZ,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6133,Television_Total Number_BA,National,BA,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6134,BCG Vaccine_Share of Population_AT,National,AT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6135,MCV2 Vaccine_Share of Population_DJ,National,DJ,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose,NaN,...,0.81,0.81,0.81,0.81,0.6,0.48,NaN,NaN,NaN,NaN


In [6]:
# Drop duplicate - air conditioners in the US
all_tech_country.drop(all_tech_country[
    (all_tech_country['Country Name'] == "United States") & 
    (all_tech_country['Technology Name'] == "Home Air Conditioning") & 
    (all_tech_country['Metric'] == 'Share of Households')
].index, inplace=True)

#### Create dataframe of only the descriptors

In [7]:
tech_timeseries_sum = all_tech_country.iloc[:, :9]
tech_timeseries_sum

,ID,Spatial Scale,Region,Country Name,Technology Name,Metric,Unit,Data Source,Variable
0,Sugar Output_Annual Production_JM,National,JM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
1,Sugar Output_Annual Production_BB,National,BB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
2,Sugar Output_Annual Production_CU,National,CU,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
3,Beer_Annual Production_GB,National,GB,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer
4,Sugar Output_Annual Production_MU,National,MU,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
...,...,...,...,...,...,...,...,...,...
6132,Crop Harvester_Total Number_UZ,National,UZ,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester
6133,Television_Total Number_BA,National,BA,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions
6134,BCG Vaccine_Share of Population_AT,National,AT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine
6135,MCV2 Vaccine_Share of Population_DJ,National,DJ,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose


#### Create dataframe of only the time series

In [8]:
tech_timeseries_nosum = all_tech_country.iloc[:,10:]

##### Convert year column names to integers.

In [9]:
# Get current column name values and convert to integers 
num_colyears = [int(year) for year in tech_timeseries_nosum.columns.values.tolist()]

# Reapply the integer column names onto the df
tech_timeseries_nosum.columns = num_colyears

### Data Cleaning

In [10]:
## Ensure that data is clean

# Define the values to replace with NaN
replace_values = {'...': np.nan, '... ': np.nan, '…': np.nan, '… ': np.nan, '··· ': np.nan, '···': np.nan, 'W': np.nan, ' ': np.nan, '-- ': np.nan, '--': np.nan}

# Replace the values with NaN
tech_timeseries_nosum = tech_timeseries_nosum.replace(replace_values)

#turn all non-nan values into float
for col in tech_timeseries_nosum.columns:
    try:
        # Convert each value to float, handling commas
        tech_timeseries_nosum[col] = tech_timeseries_nosum[col].apply(lambda x: float(str(x).replace(',', '')))
    except ValueError as e:
        print("Error converting value to float.")
        print("Error message:", e)

# Turn all non-nan values into float
tech_timeseries_nosum = tech_timeseries_nosum.astype(float)



#### Combine summary and time series dataframes and clean

In [11]:
# Create dataframe combining the cleaned time series and data description dataframes
all_tech_country_clean = pd.concat([tech_timeseries_sum, tech_timeseries_nosum], axis=1)


#### Fix country names
- ensure alignment
- switch ISO2 to ISO3 codes.


In [12]:
# List country names for alignment.
replacements = {
    'Phillipines': 'Philippines',
    'Phillipine': 'Philippines',
    'Kyrgistan': 'Kyrgyzstan',
    'Yugoslav SFR': 'Yugoslavia',
    'Southern Vietnam': 'South Vietnam'
}

for old_name, new_name in replacements.items():
    all_tech_country_clean['Country Name'].replace(old_name, new_name, inplace=True) 

In [13]:
all_tech_country_clean.rename(columns={'Region': 'Country Code'}, inplace=True)


## Convert ISO2 code to ISO3 code for countries
country_names = all_tech_country_clean["Country Name"].tolist()

new_codes = countrycode(country_names, 
                        origin = "country.name", 
                        destination = "iso3c")


all_tech_country_clean['Country Code'] = countrycode(all_tech_country_clean["Country Name"].tolist(),
                                                  origin='country.name',
                                                  destination='iso3c')

#### Remove global values.

In [14]:
# Create dataframe without global values
all_tech_country_clean_noworld = all_tech_country_clean[~all_tech_country_clean['Country Name'].str.contains('World')]




In [15]:
all_tech_country_clean_noworld

,ID,Spatial Scale,Country Code,Country Name,Technology Name,Metric,Unit,Data Source,Variable,1701,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Sugar Output_Annual Production_JM,National,JAM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sugar Output_Annual Production_BB,National,BRB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sugar Output_Annual Production_CU,National,CUB,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beer_Annual Production_GB,National,GBR,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sugar Output_Annual Production_MU,National,MUS,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,Crop Harvester_Total Number_UZ,National,UZB,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6133,Television_Total Number_BA,National,BIH,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6134,BCG Vaccine_Share of Population_AT,National,AUT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6135,MCV2 Vaccine_Share of Population_DJ,National,DJI,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose,NaN,...,0.81,0.81,0.81,0.81,0.6,0.48,NaN,NaN,NaN,NaN


#### Save files - All technologies with world data and without

In [58]:
save_filename = "all_tech_clean.csv"
save_path = os.path.join(save_filename)                
all_tech_country_clean_noworld.to_csv(save_path) 

save_filename = "all_tech_with_world.csv"
save_path = os.path.join(save_filename)
all_tech_country_clean.to_csv(save_path)

# Code block to analyze each time-country time series

In [59]:
def find_last_year(df, index):
    row_values = df.loc[index]
    for year, value in reversed(list(row_values.items())):
        if pd.notnull(value) and (value!=0):
            return year
    return None

In [60]:
#Find first year function given a dataframe and index
def find_first_year(df, index):
    row_values = df.loc[index]
    for year, value in row_values.iteritems():
        if pd.notnull(value) and (value !=0):
            return year
    return None


In [61]:

# In this function, input the time series and the descriptions
def annual_to_cumulative(num_df, sum_df):
    filtered_tech_timeseries_nosum_cum = num_df.copy()
    filtered_tech_timeseries_sum_cum = sum_df.copy()

    for row in filtered_tech_timeseries_nosum_cum.index:
        # Check the value in the 'metric' column
        metric_value = filtered_tech_timeseries_sum_cum.at[row, 'Metric']

        tech_value = filtered_tech_timeseries_sum_cum.at[row, 'Technology Name']

        ## For some data, the metrics should not be converted to cumulative.
        # These include those already in cumulative terms, and market share. Also annual production is a proxy for cumulative capacity
        if ('Cumulative' in metric_value or 
            'Share' in metric_value or 
            'Annual production' in metric_value or 
            'Installed electricity capacity' in metric_value or
            'Annual Production' in metric_value or
            'Average Capacity of Unit Additions' in metric_value or
            'Computing Capacity' in metric_value):
            continue

        ## as of Oct 2 2023, there is an issue with Github data. So we use the number of units and cumulative number of
        # units distinction from IIASA data, but by technology.
        
        if ('Railroad' in tech_value or
           'Oil Refining Capacity' in tech_value or
           'Oil Pipeline' in tech_value or
           'Gas Pipeline' in tech_value or
           'Public Roads' in tech_value ):
            continue

        # there are also technologies that are measured as 'number in use'
        # steamships, telephones, televisions, radios, cellphones, crop harvesters, cars
        # For now, we consider these cumulative.

        if ('Telephone' in tech_value or
           'Canals' in tech_value or
           'Cellphones' in tech_value or
           'Crop Harvester' in tech_value or
           'Home Air Conditioning' in tech_value or
           'Jet Aircraft' in tech_value or
           'Nuclear Weapons' in tech_value or
           'Radio' in tech_value or
           'Steamships' in tech_value or
           'Television' in tech_value):
            continue
        print(tech_value)


        # Update the value in the 'metric' column
        filtered_tech_timeseries_sum_cum.at[row, 'Metric'] = 'Cumulative Calculation'

        # Find last year in data time series
        last_year_tech = find_last_year(num_df, row)
        try:
            length = filtered_tech_timeseries_nosum_cum.columns.get_loc(last_year_tech)

            # Iterate over each column (year) starting from the second column
            for col in filtered_tech_timeseries_nosum_cum.columns[1:length + 1]:

                if col not in filtered_tech_timeseries_nosum_cum:
                    continue

                current_val = filtered_tech_timeseries_nosum_cum.at[row, col]
                prev_val = filtered_tech_timeseries_nosum_cum.at[row, col - 1]

                if np.isnan(current_val):
                    filtered_tech_timeseries_nosum_cum.at[row, col] = prev_val

                elif np.isnan(prev_val) and not np.isnan(current_val):
                    filtered_tech_timeseries_nosum_cum.at[row, col] = current_val
                else:
                    filtered_tech_timeseries_nosum_cum.at[row, col] = current_val + prev_val
        except Exception as e:
            print(f"Error occurred for index {row}: {e}")
            print(f"Details: {str(e)}")


    return filtered_tech_timeseries_sum_cum, filtered_tech_timeseries_nosum_cum

##### Apply annual to cumulative calculation

In [62]:

# Apply annual to cumulative conversions to the data 

# Dataframes to convert
tech_time_series = all_tech_country_clean_noworld.iloc[:,10:]
tech_summary = all_tech_country_clean_noworld.iloc[:, 0:9]


## Apply annual to cumulative calculation
cum_tech_growth_sum, cum_tech_growth = annual_to_cumulative(tech_time_series, 
                                                            tech_summary)

# Combine two dataframes (summary and time series)
cum_tech_growth_all = pd.concat([cum_tech_growth_sum, cum_tech_growth], axis=1)

#cum_tech_growth_all



Telegraph Traffic
Postal Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Telegraph Traffic
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Passenger Vehicles
Telegraph Traffic
Passenger Vehicles
Postal Traffic
Telegraph Traffic
Telegraph Traffic
Postal Traffic
Postal Traffic
Passenger Vehicles
Passenger Vehicles
Postal Traff

In [63]:
cum_tech_growth_all

,ID,Spatial Scale,Country Code,Country Name,Technology Name,Metric,Unit,Data Source,Variable,1702,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Sugar Output_Annual Production_JM,National,JAM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sugar Output_Annual Production_BB,National,BRB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sugar Output_Annual Production_CU,National,CUB,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beer_Annual Production_GB,National,GBR,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sugar Output_Annual Production_MU,National,MUS,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,Crop Harvester_Total Number_UZ,National,UZB,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6133,Television_Total Number_BA,National,BIH,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6134,BCG Vaccine_Share of Population_AT,National,AUT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6135,MCV2 Vaccine_Share of Population_DJ,National,DJI,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose,NaN,...,0.81,0.81,0.81,0.81,0.6,0.48,NaN,NaN,NaN,NaN


### Save CSV

In [64]:
cum_tech_growth_all

save_filename = "all_tech_clean_cumulative_Updated.csv"
save_path = os.path.join(save_filename)                
cum_tech_growth_all.to_csv(save_path) 